In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

import torch 
import os
import shutil
import random
from PIL import Image

### Data preparation

In [ ]:
cwd = os.getcwd()
print(f'Current Working Directory : {cwd}')

Current Working Directory : /content


In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d parot99/face-mask-detection-yolo-darknet-format
!unzip /content/face-mask-detection-yolo-darknet-format.zip

In [ ]:
def create_dir(root_dir, sub_dir):
    try:
        path = os.path.join(root_dir, sub_dir)
        os.makedirs(path)
        print(f'Directory "{path}" Created')
    except FileExistsError:
        print('Directory Already Exists') 

In [ ]:
sub_dirs = [
    'dataset/images/train', 
    'dataset/images/val', 
    'dataset/images/test',
    'dataset/labels/train', 
    'dataset/labels/val',
    'dataset/labels/test'
]

for sub_dir in sub_dirs:
    create_dir(cwd, sub_dir)

In [ ]:
import os
import shutil

train_folder = '/content/mask-dataset/images/test'
img_folder = '/content/dataset/images/val'
txt_folder = '/content/dataset/labels/val'

for filename in os.listdir(train_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        shutil.move(os.path.join(train_folder, filename), os.path.join(img_folder, filename))
    elif filename.endswith('.txt'):
        shutil.move(os.path.join(train_folder, filename), os.path.join(txt_folder, filename))


In [ ]:
import os
len(os.listdir('/content/dataset/images/val'))

1035

In [ ]:
# !zip -rq mask_dataset.zip mask_dataset

### Model training

In [ ]:
!cp -r /content/drive/MyDrive/mask_dataset.zip /content
!unzip mask_dataset.zip

In [ ]:
%cd /content/drive/MyDrive/face_mask
# !git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt

In [5]:
! python train.py --batch 32 --epochs 30\
                  --data data/dataset.yaml --weights yolov5s.pt --cache ram

train: weights=yolov5s.pt, cfg=, data=data/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degree

### Model Inference

In [ ]:
! python 'yolov5/detect.py' --weights 'runs/train/exp/weights/best.pt' --source 'dataset/images/test'